In [1]:
from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = "all" 
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [105]:
###########################################################################
# data pre-processing
# produce mask(foreground & background) and marker(union of disconnected cell) and weight map(blance the frequency)
# both are binary image
###########################################################################
root_path = 'DIC-C2DH-HeLa'
sequence = '02'
num_of_images = 84
k = 0.4

In [ ]:
###########################################################################
# masks
# just transfrom the ground truth masks to binary images
###########################################################################
for index in range(num_of_images):
    img = cv2.imread(f'{root_path}/{sequence}_ST/SEG/man_seg{index:03}.tif', -1)
    img = img.astype(np.uint8)
    _, res = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY)
    cv2.imshow(f'img{index}', res)
    _ = cv2.imwrite(f'{root_path}/{sequence}_generated_masks/mask{index:03}.tif', res)
    if cv2.waitKey(1) & 0XFF == ord('q'):
        break

cv2.destroyAllWindows()

In [ ]:
###########################################################################
# markers
# deal with each cell individually for each image
# each cell is eroded by a circular kernel, with diameter dSE = (1 - k) * dmax
# dmax is a diameter of the maximal disk included in the cell mask
###########################################################################
for index in range(num_of_images):
    img = cv2.imread(f'{root_path}/{sequence}_ST/SEG/man_seg{index:03}.tif', -1)
    # initialize a black image
    res = np.zeros((img.shape), dtype = np.uint8)
    # deal with each cell
    for label in np.unique(img):
        if label == 0: # ignore the background
            continue
        mask = ((label == img) * 1).astype(np.uint8)
        _, mask = cv2.threshold(mask, 0, 255, cv2.THRESH_BINARY)

        # find the maximal disk
        dist = cv2.distanceTransform(mask, cv2.DIST_L2, 3)
        dmax = cv2.minMaxLoc(dist)[1]
        # compute the dSE
        dSE = int((1 - k) * dmax)
        # get a circle structure and erode
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (dSE,dSE))
        mask = cv2.erode(mask, kernel)

        # select only the largest connected component(because the erotion will split(erode) some cell to multible component)
        nb, cc = cv2.connectedComponents(mask, connectivity=4)
        if nb != 1:
            max_size = 0
            max_cc = 0
            for i in range(1, nb):
                component = (cc == i)*1
                if (np.sum(component)) > max_size:
                    max_size = np.sum(component)
                    max_cc = i
            mask = ((max_cc == cc)*1).astype(np.uint8)
     
        # add each eroded cell to the final image
        res = cv2.add(res, mask)
    # transfrom to binary image
    _, res = cv2.threshold(res, 0, 255, cv2.THRESH_BINARY)

    cv2.imshow(f'img{index}', res)
    _ = cv2.imwrite(f'{root_path}/{sequence}_generated_markers/marker{index:03}.tif', res)
    if cv2.waitKey(1) & 0XFF == ord('q'):
        break

cv2.destroyAllWindows()

In [106]:
###########################################################################
# weight maps
# formula: W(x) = Wc + W0*exp(- (d1(x) + d2(x))²/2σ²)
# Wc: balanced class frequency map
# d1: the distance to the border of the nearest cell
# d2: the distance to the border of the second nearesr cell
###########################################################################

# parameter
w0 = 10
sigma = 5

for index in range(num_of_images):
    img = cv2.imread(f'{root_path}/{sequence}_ST/SEG/man_seg{index:03}.tif', -1)

    # compute balanced class frequency map
    w_0 = np.sum(img > 0)/img.size
    w_1 = 1 - w_0
    wc = np.where(img == 0, w_0, w_1)

    # compute d1 and d2
    distance_map = np.zeros((img.shape[0], img.shape[1], np.max(img)))
    for label in range(1, np.max(img)+1):
        mask = ((img == label)*1).astype(np.uint8)
        distance_map[:, :, label-1] = cv2.distanceTransform(1 - mask, cv2.DIST_L2, 3)
    distance_map = np.sort(distance_map, axis=2)
    d1 = distance_map[:, :, 0]
    d2 = distance_map[:, :, 1]
    weight_map = wc + w0 * np.exp(-((d1+d2)**2)/(2 * sigma * sigma))
    _ = np.save(f'{root_path}/{sequence}_weight_map/weight_map{index:03}.npy', weight_map)

In [4]:
last = cv2.imread('PhC-C2DL-PSC/01/t030.tif', -1)
_, last = cv2.threshold(last, 170, 255, cv2.THRESH_BINARY)

img = cv2.imread('PhC-C2DL-PSC/01/t001.tif', 1)
gray = cv2.imread('PhC-C2DL-PSC/01/t001.tif', -1)

_, gray = cv2.threshold(gray, 170, 255, cv2.THRESH_BINARY)

_, contours, _ = cv2.findContours(gray, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)


img = cv2.fillPoly(gray, [contours[10]], 120)

mask = np.full((576,720), 120)
res = ((mask == img)*1).astype(np.uint8)qq
_, res = cv2.threshold(res, 0, 255, cv2.THRESH_BINARY)
area = np.sum(res)/255

overlap = cv2.bitwise_and(res, last)

area_overlap = np.sum(overlap)/255

area_overlap/area

_, overlap = cv2.threshold(overlap, 0, 255, cv2.THRESH_BINARY)


cv2.imshow('asd', overlap)
_ = cv2.waitKey(0)
cv2.destroyAllWindows()


0.6981132075471698